In [1]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage, ToolMessage
from langchain_core.tools import tool
from langchain_core.tools.structured import StructuredTool
from langchain_core.documents import Document
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.llm import LLMChain
from rich import print
import random
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import (
    UnstructuredMarkdownLoader,
    DirectoryLoader,
    PythonLoader,
)
from langchain_chroma import Chroma
import chromadb
from langchain_ollama import OllamaEmbeddings, ChatOllama
from langchain_google_genai import ChatGoogleGenerativeAI
from pydantic import BaseModel, Field
from langchain_core.tools import StructuredTool
from typing import List, Dict
from glob import glob

load_dotenv()

c:\Users\roshan.yadav\AppData\Local\anaconda3\envs\rag\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
model = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
model_embedding = OllamaEmbeddings(model="nomic-embed-text:latest")
# vectore_store = Chroma(embedding_function=model_embedding)
persistent_client = chromadb.PersistentClient()

In [1]:
tools: Dict[str, StructuredTool] = []

NameError: name 'Dict' is not defined

In [3]:
def project_agent(agent_name: str, query: str) -> str:
    accessable_tools = []
    project_hub_agent_tools = Chroma(
        client=persistent_client,
        collection_name="project_hub_agent_tools",
        embedding_function=model_embedding,
    )

    finalQuery = f"Which tool should I use to respond to the query: \"{query}\""
    print(f"{finalQuery=}")
    validTools = project_hub_agent_tools.similarity_search_with_score(finalQuery)
    for function in related_tools:
        tool_name = function.metadata.get("func_name")
        accessable_tools.append(tools[tool_name])
    print(f"{agent_name=}\n{query=}")
    return "ok"

In [4]:
def embedd_all_files(project_path: str, collection_name: str):
    markdown_loader = DirectoryLoader(
        project_path,
        glob="**/*.md",
        loader_cls=UnstructuredMarkdownLoader,
        show_progress=True,
        use_multithreading=True,
    )

    python_loader = DirectoryLoader(
        project_path,
        glob="**/*.py",
        loader_cls=PythonLoader,
        show_progress=True,
        use_multithreading=True,
    )
    docs = markdown_loader.load() + python_loader.load()
    doc_sources = [doc.metadata["source"] for doc in docs]
    print(doc_sources)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    smaller_documents = text_splitter.split_documents(docs)
    vector_store_from_client = Chroma(
        client=persistent_client,
        collection_name=collection_name,
        embedding_function=model_embedding,
    )
    vector_store_from_client.add_documents(documents=smaller_documents)

In [5]:
def create_project_agent(agent_name: str, agent_description: str):
    class ProjectAgentInput(BaseModel):
        query: str = Field(description="A specific query related to the project.")

    agent = StructuredTool.from_function(
        func=lambda query: project_agent(agent_name, query),
        name=agent_name,
        description=agent_description,
        args_schema=ProjectAgentInput,
    )

    return agent

In [6]:
vector_store_from_client = Chroma(
    client=persistent_client,
    collection_name="project_hub_agent_tools",
    embedding_function=model_embedding,
)
for project_path in glob("./Github/*"):
    project_name = project_path.split("\\")[-1]
    agent_name = project_name.replace("-", "_").lower() + "_project_agent"
    tool_short_desc = f"Provides details about {project_name} project."
    about_project_filepath = glob(f"{project_path}/*.description")[0]
    with open(about_project_filepath, "r") as about_file:
        about_project = about_file.read().replace("tool", "project")
    agent_description = f"{tool_short_desc.capitalize()}\n\n <AboutProject>{about_project.capitalize()}</AboutProject>"
    tools[agent_name] = create_project_agent(agent_name, agent_description)
    embedd_all_files(project_path, agent_name)

AvailableTools = {tool.name: tool for tool in tools}
toolDoc = [
    Document(
        page_content=f"Tool Name: {tool.name}\n\nTool Description:\n{tool.description}",
        metadata={"func_name": tool.name},
    )
    for tool in tools
]
vector_store_from_client.add_documents(documents=toolDoc)

100%|██████████| 5/5 [00:00<00:00, 945.05it/s]


[
    'Github\\Auto_Dictionary_database_creator_tool\\README.md',
    'Github\\Auto_Dictionary_database_creator_tool\\DictionaryDatabase_Advanced_scrape_without_clicks\\custom_funct
ion.py',
    'Github\\Auto_Dictionary_database_creator_tool\\DictionaryDatabase_inerable_gui_based_clicks\\main.py',
    'Github\\Auto_Dictionary_database_creator_tool\\DictionaryDatabase_Advanced_scrape_without_clicks\\Google_Trans
late_Scrape.py',
    'Github\\Auto_Dictionary_database_creator_tool\\DictionaryDatabase_inerable_gui_based_clicks\\test\\test.py',
    'Github\\Auto_Dictionary_database_creator_tool\\Fetch_dictionary_to_json\\fetch_wordlist_to json.py'
]

100%|██████████| 1/1 [00:00<00:00, 501.59it/s]


['Github\\Plant-monitoring-system\\README.md', 'Github\\Plant-monitoring-system\\main.py']

['93f71973-71a3-453e-9967-395f74603121',
 '2d5c7860-7976-43c0-8d3e-d4ca18cc3e44']

In [7]:
for tool in tools:
    resp = tool.invoke("Hii")

agent_name='auto_dictionary_database_creator_tool_project_agent'
query='Hii'
resp='yes'

agent_name='plant_monitoring_system_project_agent'
query='Hii'
resp='no'

In [8]:
@tool
def project_hub_agent(query: str):
    prompt = ChatPromptTemplate(
        [
            SystemMessage(
                content="You are ProjectHub, an assistant that knows all of Roshan Yadav's projects, helping to describe, manage, debug, and enhance them while providing technical and communication support."
            ),
            HumanMessage(
                content="Could you please list all the projects I have developed?"
            ),
        ]
    )

C:\Users\roshan.yadav\AppData\Local\Temp\ipykernel_2956\600197521.py:1: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  @tool


ValidationError: 1 validation error for ProjectAgentInput
  Input should be a valid dictionary or instance of ProjectAgentInput [type=model_type, input_value=<function project_hub_agent at 0x000001E85167C5E0>, input_type=function]
    For further information visit https://errors.pydantic.dev/2.9/v/model_type

In [ ]:
prompt = ChatPromptTemplate(
    [
        SystemMessage(
            content="You are ProjectHub, an assistant that knows all of Roshan Yadav's projects, helping to describe, manage, debug, and enhance them while providing technical and communication support."
        ),
        HumanMessage(
            content="Could you please list all the projects I have developed? i not need "
        ),
    ]
)